# langgraph
'langgraph.ipynb'

In [2]:
import os
from dotenv import load_dotenv

# load_dotenv()
# if you are running this in a Jupyter notebook, use this instead:
load_dotenv(override=True)

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool

# Supabase tool setup - will be used later
# from supabase_tool import SupabaseTool
# from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_community.agent_toolkits import create_conversational_retrieval_agent

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [ ]:
# 1. LLM 설정
# Re Act 에이전트는 도구 호출(tool-calling) 기능을 활용하는 LLM을 사용
# OpenAI 모델을 사용하며, 특정 함수를 호출할 수 있도록 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. Re Act 프롬프트
# 에이전트의 시스템 프롬프트를 정의합니다.
# Re Act 에이전트가 어떤 역할을 수행하고 어떻게 행동해야 하는지 지시
system_prompt = (
    "You are a fitness and diet agent. "
    "Your main task is to assist the user with their workout and diet plans. "
    "Use the tools provided to access and manage user data. "
    "If the user asks about their workout or diet, use the appropriate tools to provide a detailed response. "
    "If the user's query is not related to fitness or diet, respond politely and inform them of your purpose."
)

# ChatPromptTemplate은 LLM 프롬프트를 구성하는 데 사용됩니다.
# 이 템플릿은 시스템 메시지와 사용자 입력, 그리고 도구 호출에 필요한 메시지 플레이스홀더를 포함
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 3. 도구 정의
# 에이전트가 사용할 도구를 정의
# 지금은 간단한 더미 도구를 추가. 추후에 Supabase 관련 기능으로 확장 예정
@tool
def get_workout_history(user_id: str) -> str:
    """Gets the workout history for a specific user."""
    # This is a dummy function. It will be replaced with Supabase integration.
    return "You have not recorded any workout history yet."

@tool
def get_nutrition_log(user_id: str) -> str:
    """Gets the nutrition log for a specific user."""
    # This is a dummy function. It will be replaced with Supabase integration.
    return "You have not recorded any nutrition log yet."

tools = [get_workout_history, get_nutrition_log]

# 4. LCEL 체인 결합
# LLM, 프롬프트, 도구를 결합하여 에이전트의 기본적인 LCEL(LangChain Expression Language) 체인을 구축합니다.
# 이 체인은 프롬프트, LLM, 도구의 순서로 구성되어, 사용자의 입력이 들어오면 프롬프트에 따라 LLM이 추론하고, 필요시 도구를 호출
agent = prompt | llm.bind_tools(tools)

# 5. 테스트
# 테스트용 입력
input_text = "What is my workout history?"

# 에이전트 실행
# agent_scratchpad를 빈 리스트로 추가하여 전달
output = agent.invoke({
    "input": input_text, 
    "chat_history": [], 
    "agent_scratchpad": [] 
})

# 결과 출력
print(output)

content='Please provide me with your user ID so I can access your workout history.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 157, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CJBlkH4OvjsGJMhC9zH2PTJoRMNt9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f1fe0116-5647-4694-8b86-0be3be11d875-0' usage_metadata={'input_tokens': 157, 'output_tokens': 16, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


: 

: 

In [ ]:
import os
# 현재 작업 디렉토리 확인
print(os.getcwd())

/Users/jaehyuntak/Desktop/project_at25-09-15/pjt-final-gaida/langgraph-agent


: 

: 

In [ ]:
# langgraph.ipynb
# 기존 코드에 다음 부분을 추가하거나 수정합니다.

# 1. 필요한 모듈 임포트
# 기존 @tool로 정의된 더미 함수 대신,
# 이제 supabase_tools.py 파일에서 실제 도구들을 가져옵니다.
from supabase_tools import get_workout_history, add_workout_session
from langchain_core.agents import AgentFinish
from langgraph.graph import StateGraph
import operator

import sys
module_path = '/Users/jaehyuntak/Desktop/project_at25-09-15/pjt-final-gaida/langgraph-agent/'
sys.path.append(module_path)

from state import State # langgraph.ipynb 파일 상단에 state.py에서 import

# 2. 도구 정의
# 이제 직접 만든 Supabase 도구들을 사용합니다.
tools = [get_workout_history, add_workout_session]

# 3. Agent Executor 생성 (새로운 부분)
from langchain.agents import AgentExecutor, create_tool_calling_agent
# Agent Executor는 에이전트의 '추론'과 '도구 호출' 과정을 관리합니다.
# create_tool_calling_agent 함수는 LLM, 도구, 프롬프트를 결합하여 에이전트를 만듭니다.
agent_executor = create_tool_calling_agent(llm, tools, prompt)

# 4. Agent Graph 정의 (새로운 부분)
# LangGraph를 사용하여 에이전트의 상태와 흐름을 정의합니다.
workflow = StateGraph(State)

# 5. Graph에 노드 추가
# 노드 정의: 에이전트 추론 노드와 도구 호출 노드를 정의합니다.
def run_agent(state: State):
    """Agent node: responsible for making a decision."""
    # 에이전트 실행
    return {"agent_outcome": agent_executor.invoke(state)}

def run_tools(state: State):
    """Tool node: responsible for executing tools."""
    tool_input = state["agent_outcome"].tool_input
    # 이 부분은 실제 tool 실행 로직으로 대체됩니다.
    # 지금은 임시로 더미 함수를 사용합니다.
    if state["agent_outcome"].tool == "get_workout_history":
        result = get_workout_history.invoke({"user_id": tool_input["user_id"]})
    elif state["agent_outcome"].tool == "add_workout_session":
        result = add_workout_session.invoke(tool_input)
    else:
        result = f"Unknown tool: {state['agent_outcome'].tool}"
    
    # 실행 결과를 상태에 저장합니다.
    return {"result": result}


workflow.add_node("agent", run_agent)
workflow.add_node("tools", run_tools)

# 6. Graph에 엣지 추가
# 엣지 정의: 에이전트의 다음 단계를 결정하는 조건부 로직을 추가합니다.
def should_continue(state):
    """Determines which node to run next."""
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "tools"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "end": "end",
        "tools": "tools",
    },
)
workflow.add_edge("tools", "agent")

workflow.set_entry_point("agent")

app = workflow.compile()

# 7. 실행 테스트
# 이제 실제 Supabase 도구를 사용하는 에이전트 그래프를 실행할 수 있습니다.
# (이 부분은 사용자 입력과 데이터베이스 연결에 따라 달라집니다.)
input_data = {"input": "What is my workout history? My user ID is 'test-user-123'", "chat_history": []}
result = app.invoke(input_data)
print(result)

ImportError: cannot import name 'State' from 'state' (/Users/jaehyuntak/Desktop/project_at25-09-15/pjt-final-gaida/langgraph-agent/state.py)

: 

: 